# Cálculo de los invariantes de los tensores de esfuerzos medios (hidrostáticos) y desviadores

|Quién | Fecha | Qué hizo |
| ---  | ---   | ---      |
|Diego Andrés Alvarez Marín, <daalvarez@unal.edu.co>  | Año 2020 | Primer código |
| Nathalie Pineda Pardo | Año 2023  | Adaptando el código de MAXIMA a Python  |
|Diego Andrés Alvarez Marín, <daalvarez@unal.edu.co>  | Septiembre 6, 2024 | Mejorando los comentarios |
|Michael Heredia Pérez, <mherediap@unal.edu.co>  | Junio 1, 2025 | Incluyendo explicaciones y mejorando continuidad |

$$
\newcommand{\ve}[1]{{\boldsymbol{#1}}}
\newcommand{\ma}[1]{{\boldsymbol{#1}}}
\newcommand{\hati}{\ve{\hat i}}
\newcommand{\hatj}{\ve{\hat j}}
\newcommand{\hatk}{\ve{\hat k}}
\newcommand{\hatr}{\ve{\hat r}}
\newcommand{\hatt}{\ve{\hat \theta}}
\newcommand{\hatp}{\ve{\hat \varphi}}
$$

Recordemos que resolviendo el determinante $\det\left( \boldsymbol{\underline{\underline{\sigma}}} - \sigma_n \boldsymbol{I}\right) = 0$, llegamos a:

\begin{multline*}
    \left( \sigma_x - \sigma_n \right) \left[ (\sigma_y - \sigma_n)(\sigma_z - \sigma_n) - \tau_{yz}^2 \right] \\
    - \tau_{xy}\left[ \tau_{xy}(\sigma_z - \sigma_n) - \tau_{yz}\tau_{xz} \right] \\
    + \tau_{xz} \left[\tau_{xy}\tau_{yz} - (\sigma_y - \sigma_n)\tau_{xz}\right] = 0;
\end{multline*}

Lo cual simplificamos en lo que se conoce como la ecuación característica:
\begin{equation*}
    -\sigma_n^3 + I_1 \sigma_n^2 - I_2 \sigma_n + I_3 = 0
\end{equation*}


Cuyos coeficientes son los llamados **invariantes de esfuerzos**:
\begin{align*}
    \Theta := I_1 &= \operatorname{tr}(\underline{\underline{\sigma}}) \\
    I_2 &= \frac{1}{2}((\operatorname{tr}(\underline{\underline{\sigma}}))^2 - \operatorname{tr}(\underline{\underline{\sigma}}^2)) \\
    I_3 &= \det(\underline{\underline{\sigma}})
\end{align*}

Los invariantes son útiles cuando se necesita crear descripciones del comportamiento de los materiales, ya que, por ejemplo, un material isótropo no sabe qué sistema de coordenadas se ha usado para describir su comportamiento.

## 1. Preliminares del código

In [1]:
# Importamos los módulos de cálculo simbólico necesarias.
from sympy import symbols, init_printing, latex, expand, diag, det, trace, Matrix, factor, simplify

#Para imprimir bonito
init_printing()
from IPython.display import Math
def imprimir (texto1, variable, texto2=""):
    return Math(texto1 +  rf'{latex(variable)}' + texto2)

# Definimos las variables necesarias para comenzar.
sigmax, sigmay, sigmaz,txy, txz, tyz = symbols('sigma_x, sigma_y, sigma_z, tau_xy, tau_xz, tau_yz')
E, nu                                = symbols('E, nu')

## 2. Esfuerzos medios o hidrostáticos

Los esfuerzos medios o hidrostáticos, definidos como $\sigma_M$ están relacionados con la dilatación del sólido. Se calcula como:
\begin{equation}
    \sigma_M = \frac{\sigma_x + \sigma_y + \sigma_z}{3} =\frac{I_1}{3}
\end{equation}

Lo anterior en el sistema de coordenadas canónico$\{\boldsymbol{\hat{i}}, \boldsymbol{\hat{j}}, \boldsymbol{\hat{k}}\}$}, en el sistema de coordenadas dado por la direcciones principales $\{\boldsymbol{\hat{n}_1}, \boldsymbol{\hat{n}_2}, \boldsymbol{\hat{n}_3}\}$ sería
\begin{equation}
    \sigma_M = \frac{\sigma_1 + \sigma_2 + \sigma_3}{3} =\frac{I_1}{3}
\end{equation}

El tensor de esfuerzos hidrostáticos viene dado por:
\begin{equation}
\underline{\underline{\boldsymbol{\sigma}}}_{\text{hidr}} =
\begin{bmatrix}
    \sigma_M & 0 & 0 \\
    0 & \sigma_M & 0 \\
    0 & 0 & \sigma_M
\end{bmatrix}
\end{equation}

In [2]:
# Simbólicamente, el esfuerzo medio o hidrostático
sM = symbols('sigma_M')

# Simbólicamente, la matriz de esfuerzos medios o hidrostáticos.
sigma_hidr = diag(sM, sM, sM)

Los invariantes de esfuerzo del tensor de esfuerzos medios o hidrostáticos son:
\begin{equation*}
		\widetilde{I}_1 := I_1 = 3\sigma_M = \operatorname{tr}(\underline{\underline{\sigma}}) \qquad
		\widetilde{I}_2 := \frac{I_1^2}{3} = 3\sigma_M^2 \qquad
		\widetilde{I}_3 := \frac{I_1^3}{27} = \sigma_M^3.
\end{equation*}

A continuación, los calculamos:

In [3]:
I1 = trace(sigma_hidr)
I2 = (trace(sigma_hidr)**2 - trace((sigma_hidr)**2))/2
I3 = det(sigma_hidr)

In [4]:
imprimir("I_1 = ", I1)

<IPython.core.display.Math object>

In [5]:
imprimir("I_2 = ", I2)

<IPython.core.display.Math object>

In [6]:
imprimir("I_3 = ", I3)

<IPython.core.display.Math object>

## 3. Verificación de que los esfuerzos desviadores no afectan el volumen del sólido

Los esfuerzos desviadores son responsables de las distorsiones del sólido. Estos se definen como:
\begin{align*}
    s_x &:= \sigma_x - \sigma_M  \\
    s_y &:= \sigma_y - \sigma_M  \\
    s_z &:= \sigma_z - \sigma_M
\end{align*}

Calculamos sus deformaciones asociadas mediante la Ley de Hooke (ecuaciones de Lamé):
\begin{align*}
  \varepsilon_x & = \frac{1}{E} \left(\sigma_x - \nu\left(\sigma_y + \sigma_z\right)\right)\\
  \varepsilon_y & = \frac{1}{E} \left(\sigma_y - \nu\left(\sigma_x + \sigma_z\right)\right)\\
  \varepsilon_z & = \frac{1}{E} \left(\sigma_z - \nu\left(\sigma_x + \sigma_y\right)\right)
\end{align*}

In [7]:
# esfuerzos hidrostáticos
sM = (sigmax + sigmay + sigmaz)/3

# esfuerzos desviadores
sx = sigmax - sM
sy = sigmay - sM
sz = sigmaz - sM

# Calculamos las deformaciones
ex = (1/E) * (sx - nu*(sy + sz))
ey = (1/E) * (sy - nu*(sx + sz))
ez = (1/E) * (sz - nu*(sx + sy))

# Verificamos que la dilatación cúbica es nula (e=0)
e = expand(ex + ey + ez)

# Mostramos en pantalla
imprimir("e(x,y,z) = ", e)

<IPython.core.display.Math object>

## 4. Descomposición del tensor de esfuerzos en sus componentes hidrostática y desviadora

Dadas las definiciones anteriores de los esfuerzos medios o hidrostáticos, y los desviadores, podemos hacer la siguiente construcción matricial del tensor de esfuerzos:

\begin{equation}
\underbrace{
 \begin{bmatrix}
	\sigma_x  & \tau_{xy} & \tau_{xz} \\
	\tau_{xy} & \sigma_y  & \tau_{yz} \\
	\tau_{xz} & \tau_{yz} & \sigma_z
 \end{bmatrix}}_{\boldsymbol{\sigma}} =
\underbrace{%
 \begin{bmatrix}
	\sigma_M  & 0         & 0 \\
	0         & \sigma_M  & 0 \\
	0         & 0         & \sigma_M
\end{bmatrix}}_{\boldsymbol{\sigma}_{\text{hidr}}}
+
\underbrace{%
 \begin{bmatrix}
        s_x & \tau_{xy} & \tau_{xz} \\
        \tau_{xy} & s_y & \tau_{yz} \\
        \tau_{xz} & \tau_{yz} & s_z
 \end{bmatrix}}_{\boldsymbol{\sigma}_{\text{desv}}}
\end{equation}

In [8]:
# Se define el tensor de esfuerzos hidrostáticos
sigma_hidr = diag(sM, sM, sM)

# Se define el tensor de esfuerzos desviadores
sigma_desv = Matrix([
    [sx, txy, txz],
    [txy, sy, tyz],
    [txz, tyz, sz]
    ])

# Se define el tensor de esfuerzos
sigma = sigma_hidr + sigma_desv

## 4. Cálculo de los invariantes de esfuerzos

Calculamos los invariantes del tensor de esfuerzos ya que encontraremos más adelante simplificaciones en función de estos.

In [9]:
I1 = trace(sigma)
I2 = (trace(sigma)**2 - trace((sigma)**2))/2
I3 = det(sigma)

Se calculan de nuevo los invariantes de esfuerzos hidrostáticos, ahora estos estarán en témrinos de los esfuerzos normales y tangenciales en lugar del esfuerzo medio o hidrostático.

In [10]:
Itilde1 = trace(sigma_hidr)
Itilde2 = (trace(sigma_hidr)**2 - trace((sigma_hidr)**2))/2
Itilde3 = det(sigma_hidr)

In [11]:
imprimir(r"\tilde{I}_1 = ", Itilde1)

<IPython.core.display.Math object>

In [12]:
imprimir(r"\tilde{I}_2 = ", Itilde2)

<IPython.core.display.Math object>

In [13]:
imprimir(r"\tilde{I}_3 = ", Itilde3)

<IPython.core.display.Math object>

Se calculan los invariantes de esfuerzos desviadores:

In [14]:
J1 = trace(sigma_desv)
J2 = -(trace(sigma_desv)**2 - trace((sigma_desv)**2))/2  # OJO con el signo menos al principio
J3 = det(sigma_desv)

In [15]:
imprimir("J_1 = ", J1)

<IPython.core.display.Math object>

In [16]:
imprimir("J_2 = ", J2)

<IPython.core.display.Math object>

In [17]:
imprimir("J_3 = ", J3)

<IPython.core.display.Math object>

## 5. Verificando las expresiones de los invariantes $J_2$ y $J_3$

En el desarrollo del segundo invariante de esfuerzos desviadores $J_2$, se encontraron las siguientes relaciones:
\begin{align*}
J_2 &:= \frac{(\sigma_x - \sigma_y)^2 + (\sigma_y - \sigma_z)^2 + (\sigma_z - \sigma_x)^2}{6} + \tau_{xy}^2 + \tau_{xz}^2 + \tau_{yz}^2 \\
    &= \frac{(\sigma_1 - \sigma_2)^2 + (\sigma_2 - \sigma_3)^2 + (\sigma_3 - \sigma_1)^2}{6} \\
    &= \frac{I_1^2}{3} - I_2 \\
    &= \frac{1}{2} \left[ \operatorname{tr}(\underline{\underline{\sigma}}^2) - \frac{1}{3} \operatorname{tr}(\underline{\underline{\sigma}})^2 \right] \\
    &= \frac{1}{2} \operatorname{tr}(\underline{\underline{\sigma}}_{\text{desv}}^2) \\
    &= \frac{1}{2} \left\lVert \underline{\underline{\sigma}}_{\text{desv}} \right\rVert_F^2 \\
    &= -s_1 s_2 - s_2 s_3 - s_1 s_3 \\
    &= \frac{1}{2} \left( s_1^2 + s_2^2 + s_3^2 \right) \\
    &= \frac{1}{2} \left\lVert [s_1, \; s_2, \; s_3]^T \right\rVert^2
\end{align*}

Estas van a ser verificadas a continuación:

In [18]:
expr_J2 = ((sigmax - sigmay)**2 + (sigmay - sigmaz)**2 + (sigmaz - sigmax)**2)/6 + txy**2 + txz**2 + tyz**2
simplify(J2 - expr_J2)

0

In [19]:
expr_J2 = I1**2/3 - I2
simplify(J2 - expr_J2)

0

In [20]:
expr_J2 = (trace(sigma**2) - trace(sigma)**2/3)/2
simplify(J2 - expr_J2)

0

In [21]:
expr_J2 = trace(sigma_desv**2)/2
simplify(J2 - expr_J2)

0

$\|\boldsymbol{\sigma}_{\text{desv}}\|_F$ representa la norma de Frobenius de la matriz $\boldsymbol{\sigma}_{\text{desv}}$. La norma de Frobenius de una matriz real $A$ de tamaño $m \times n$ se define como
$\|A\|_F = \sum_{i=1}^{m} \sum_{j=1}^{n} A_{ij}^2 = \sqrt{\text{tr}(A^T A)}$ o en notación indicial como $\|A\|_F = A_{ij} A_{ij}$.


In [22]:
# se calcula la norma de Frobenius
expr_J2 = sum(sigma_desv[i,j]**2 for i in range(3) for j in range(3))/2
simplify(J2 - expr_J2)

0

En el desarrollo del tercer invariante de esfuerzos desviadores $J_3$, se encontraron las siguientes relaciones:
\begin{align*}
				J_3 &:= \frac{(2\sigma_1 - \sigma_2 - \sigma_3)(2\sigma_2 - \sigma_1 - \sigma_3)(2\sigma_3 - \sigma_1 - \sigma_2)}{27} \\
					&= I_3 - \frac{1}{3} I_1 I_2 + \frac{2}{27} I_1^3 \\
					&= \det(\underline{\underline{\sigma}}_{\text{desv}}) \\
					&= \frac{1}{3} \operatorname{tr}(\underline{\underline{\sigma}}_{\text{desv}}^3) \\
					&= s_1 s_2 s_3.
			\end{align*}

Estas van a ser verificadas a continuación:

In [23]:
expr_J3 = I3 - I1*I2/3 + 2*I1**3/27
simplify(J3 - expr_J3)

0

In [24]:
expr_J3 = det(sigma_desv)
simplify(J3 - expr_J3)

0

In [25]:
expr_J3 = trace(sigma_desv**3)/3
simplify(J3 - expr_J3)

0

## 7. Verificación en los esfuerzos desviadores principales

In [26]:
# Definimos los esfuerzos principales
sigma1, sigma2, sigma3 = symbols('sigma_1, sigma_2, sigma_3')

# En estos términos, los esfuerzos hidrostáticos principales...
sM = (sigma1 + sigma2 + sigma3)/3

# ... y los esfuerzos desviadores principales.
s1 = sigma1 - sM
s2 = sigma2 - sM
s3 = sigma3 - sM

# Definimos el tensor de esfuerzos desviadores principales
sigma_desvp = diag(s1, s2, s3)

# Calculamos los invariantes asociados a este tensor.
J1 = trace(sigma_desvp)
J2 = -(trace(sigma_desvp)**2 - trace((sigma_desvp)**2))/2  # OJO con el signo menos al principio
J3 = det(sigma_desvp)

In [27]:
imprimir("J_1 = ", J1)

<IPython.core.display.Math object>

In [28]:
imprimir("J_2 = ", J2)

<IPython.core.display.Math object>

In [29]:
imprimir("J_3 = ", J3)

<IPython.core.display.Math object>

Verificamos algunas de las expresiones anteriormente mencionadas para el invariante $J_2$.

In [30]:
expr_J2 = ((sigma1 - sigma2)**2 + (sigma2 - sigma3)**2 + (sigma1 - sigma3)**2)/6
simplify(J2 - expr_J2)

0

In [31]:
expr_J2 = -s1*s2 - s2*s3 - s1*s3
simplify(J2 - expr_J2)

0

In [32]:
expr_J2 = (s1**2 + s2**2 + s3**2)/2
simplify(J2 - expr_J2)

0

Verificamos algunas de las expresiones anteriormente mencionadas para el invariante $J_3$.

In [33]:
expr_J3 = (2*sigma1 - sigma2 - sigma3)*(2*sigma2 - sigma1 - sigma3)*(2*sigma3 - sigma1 - sigma2)/27
simplify(J3 - expr_J3)

0

In [34]:
expr_J3 = s1*s2*s3
simplify(J3 - expr_J3)

0

Fin :)